In [5]:
import os, csv
import xml.etree.ElementTree as ET
import pprint

In [6]:
cwd = os.getcwd()
print(cwd)
os.listdir(cwd)
all_cwe_types = {}
all_files = 0

C:\Users\Madhu\School\summer2021\research\Paper1


In [3]:
def parse(fileName):
    global all_files
    post_ids_with_errors = []
    post_ids_with_errors_cwecount = {}
    print("parsing " + fileName)
    iot = ET.parse(fileName)
    root = iot.getroot()
    
    errors = root[1]
    count = 0
    dict = {}
    list = []
    cweDict = {}
    for i in errors:
        
        key = i.get("id")
        if i.get("cwe") == "563":
            continue
#         if i.get("cwe") == "401":
#             print("yes 401")
        count = count + 1
        if i.get("cwe") is not None and i.get("cwe") != "563": 
            key = key + " cwe: " + i.get("cwe")
        if key in dict:
            dict[key] += 1 
        else:
            dict[key] = 1
        if i.get("cwe") is not None and i.get("cwe") != "563" and i.get("cwe") in cweDict:
            cweDict[i.get("cwe")] += 1
        else:
            cweDict[i.get("cwe")] = 1
            
        if i.get("cwe") is not None and i.get("cwe") != "563" and i.get("cwe") in all_cwe_types:
            all_cwe_types[i.get("cwe")] += 1
        else:
            all_cwe_types[i.get("cwe")] = 1
            
        loc = i[0]
        if loc.get("file") in list:
            continue
        else:
            list.append(loc.get("file"))
        fileName = loc.get("file")
        f = fileName.split("-")
        f_pid = ""
        if len(f)<2:
            continue
        if ".c" in f[1]:
            a = f[1]
            a = a.rstrip(a[-1])
            f_pid = a.rstrip(a[-1])
            post_ids_with_errors.append(a.rstrip(a[-1]))
        else:
            post_ids_with_errors.append(f[1])
            f_pid = f[1]
        
        if f_pid in post_ids_with_errors_cwecount:
            post_ids_with_errors_cwecount[f_pid] += 1
        else:
            post_ids_with_errors_cwecount[f_pid] = 1
        
        

    print("total errors: ",count)
    print(dict)
    print("(number of files): ", len(list))
    all_files = all_files + len(list)
    print("all files: ", all_files)
    print("(number of unique errors): ", len(dict))
    
    print("list of CWEs")
    sorted_cwe_dict = {}
    sorted_keys = sorted(cweDict, key=cweDict.get)
    for w in sorted_keys:
        sorted_cwe_dict[w] = cweDict[w]
    #print(sorted_cwe_dict)
    pprint.pprint(sorted_cwe_dict)
#     for p in post_ids_with_errors:
#         print(p)
    for key, value in post_ids_with_errors_cwecount.items():
        print(key, ' , ', value)
    
    print("----------------------------------------------------------------------------------")

In [20]:
def parse_2(fileName):
    cweTypes_files = {}
    cweList = []
    
    print("parsing " + fileName)
    iot = ET.parse(fileName)
    root = iot.getroot()
    errors = root[1]
    count398 = {}
    for i in errors:
        newList = []
        if i.get("cwe") == "563":
            continue
        if i.get("cwe") == "398":
            for ll in i:
                file1 = ll.get("file")
                if file1 in count398:
                    count398[file1] = count398[file1] + 1
                else:
                    count398[file1] = 1
            continue
        if i.get("cwe") is not None and i.get("cwe") != "563": 
            key = i.get("cwe")
        cweList.append(key)
        for l in i:
            file = l.get("file")
            
        if key in cweTypes_files:
            if file in cweTypes_files[key]:
                continue
            else:
                cweTypes_files[key].append(file)
        else:
            newList.append(file)
            cweTypes_files[key] = newList
    print("number of instances of 398: ")
#     for k, v in count398.items():
#         print(k, v)
    print(len(count398))
#     sum = 0    
#     for k, v in cweTypes_files.items():
#         print(k, len(v))
#         sum = sum + len(v)
#     print(fileName, sum)
        


In [21]:
parse_2("arduino/output.xml")
parse_2("raspberrypi/output.xml")
parse_2("stackoverflow/output-cpp.xml")
parse_2("stackoverflow/output-c.xml")

parsing arduino/output.xml
number of instances of 398: 
197
parsing raspberrypi/output.xml
number of instances of 398: 
21
parsing stackoverflow/output-cpp.xml
number of instances of 398: 
159
parsing stackoverflow/output-c.xml
number of instances of 398: 
45


In [23]:
def parse_3(fileName):
    cweTypes_files = {}
    cweList = []
    
#     print("parsing " + fileName)
    iot = ET.parse(fileName)
    root = iot.getroot()
    errors = root[1]
    newList = []
    newDict = {}
    for i in errors:
       
        if i.get("cwe") != "398":
            continue
        key = i.get("cwe")
        for l in i:
            file = l.get("file")
        if file in newList: 
            continue
        else: 
            newList.append(file)
            if key in newDict.keys():
                newDict[key] = newDict[key] + 1
            else:
                newDict[key] = 1
#     print(len(newList))
    sum = 0
    for k, v in newDict.items():
        sum += v
        print(k,v)
    print(sum)

In [75]:
def getCwes_with_ids(xmlFile):
    cwes_ids = {}
    iot = ET.parse(xmlFile)
    root = iot.getroot()
    errors = root[1]
    for e in errors:
        if e.get("cwe") == "398":
            continue
        key = e.get("cwe")
        for files in e:
            f = files.get("file")
            f_split = f.split("-")
            f_pid = f
            if len(f_split)>=2:
                if ".c" in f_split[1]:
                    a = f_split[1]
                    a = a.rstrip(a[-1])
                    f_pid = a.rstrip(a[-1])
                else:
                    f_pid = f_split[1]
            if key in cwes_ids.keys():
                temp = cwes_ids[key]
                if f_pid not in temp:
                    cwes_ids[key].append(f_pid)
            else:
                temp = [f_pid]
                cwes_ids[key] = temp
    return cwes_ids

In [78]:
def getQuestionViewCount(xmlFile, post_info):
    cwes_ids = {}
    parentIds_views = {}
    ids_parents = {}
    cwes_ids = getCwes_with_ids(xmlFile)
    for k, v in cwes_ids.items():
        print(k,v)
#     with open(post_info, mode='r', encoding="utf8") as f:
#         post_info = csv.DictReader(f)
#         for row in post_info:
#             parentIds_views[row['QIds']] = row['QViewCounts']
#             if row['\ufeffPostId'] not in ids_parents.keys():
#                 ids_parents[row['\ufeffPostId']] = row['ParentId']
#     for cwe, ids in cwes_ids.items():
#         sub_viewcount = 0
#         dup_parents = []
#         count = 0
#         for p in ids:
#             if "." in p:
#                 p = p[:-1]
# #             print(cwe, ids_parents[p])
# #             if ids_parents[p] in dup_parents:
# #                 print(cwe, ids_parents[p])
#             if ids_parents[p] not in dup_parents:
#                 parent_p = ids_parents[p]
#                 viewcount = parentIds_views[parent_p]
#     #             print(cwe,p,viewcount)
#                 sub_viewcount += int(viewcount)
#                 dup_parents.append(ids_parents[p])
#                 count +=1
#         print(cwe, sub_viewcount, count)

In [79]:
getQuestionViewCount("arduino/output.xml", "arduino/Post_Information.csv")

571 ['10228', '16155', '19541', '25180', '48548', '60282', '61306', '63357', '68873', '71618', '71995', '76182', '76183', '77351', '77364', '81271', '81461']
457 ['10407', '20300', '20912', '20913', '23279', '25785', '31506', '36078', '47503', '48446', '48694', '51810', '57867', '65929', '69644', '72696', '75259', '75937', '78070', '79570', '79740', '80228', '80731']
595 ['12208', '22408', '33039', '3954', '48493', '52070', '52585', '64833', '6662', '69577', '71425', '71522', '75312', '78395']
788 ['14169', '1605', '19457', '20464', '25615', '31422', '31796', '43628', '44892', '51810', '54686', '55266', '57798', '60865', '60892', '65502', '70083', '70970', '77972', '9630']
467 ['1447', '30709']
686 ['16003', '480', '80289', '81112']
561 ['17255', '19710', '32358', '42974', '58283', '9314']
401 ['18097', '19093', '48771']
683 ['18795']
664 ['201']
476 ['215']
570 ['31798', '33584', '43565', '6448', '70967', '70985', '75454', '777', '8600']
563 ['33584', '5357', '59972', '6549', '69476',

In [26]:
parse_3("arduino/output.xml")
#parse_3("raspberrypi/output.xml")
parse_3("stackoverflow/output-cpp.xml")
parse_3("stackoverflow/output-c.xml")
# print(123+17+52+39)

398 197
197
398 159
159
398 45
45


In [8]:
parse("arduino/output.xml")
# parse("iot/output.xml")
# parse("iota/output.xml")
#parse("raspberrypi/output.xml")
#parse("stackoverflow/output-cpp.xml")
#parse("stackoverflow/output-c.xml")
sorted_dict = {}
sorted_k = sorted(all_cwe_types, key=all_cwe_types.get)
for w in sorted_k:
    sorted_dict[w] = all_cwe_types[w]
pprint.pprint(sorted_dict)

print(sum(all_cwe_types.values()))

# parse("arduino/pre-supress.xml")
# parse("iot/pre-supress.xml")
# parse("iota/pre-supress.xml")
# parse("raspberrypi/pre-supress.xml")
# parse("stackoverflow/pre-supress-cpp.xml")
# parse("stackoverflow/pre-supress-c.xml")
# sorted_dict = {}
# sorted_k = sorted(all_cwe_types, key=all_cwe_types.get)
# for w in sorted_k:
#     sorted_dict[w] = all_cwe_types[w]
# pprint.pprint(sorted_dict)

# print(sum(all_cwe_types.values()))

parsing arduino/output.xml
total errors:  465
{'knownConditionTrueFalse cwe: 571': 20, 'uninitvar cwe: 457': 29, 'clarifyCondition cwe: 398': 14, 'literalWithCharPtrCompare cwe: 595': 32, 'constStatement cwe: 398': 16, 'variableScope cwe: 398': 173, 'shadowVariable cwe: 398': 62, 'arrayIndexOutOfBounds cwe: 788': 17, 'sizeofwithsilentarraypointer cwe: 467': 1, 'invalidScanfArgType_int cwe: 686': 1, 'arrayIndexOutOfBoundsCond cwe: 788': 10, 'unreachableCode cwe: 561': 4, 'memleakOnRealloc cwe: 401': 2, 'uselessAssignmentArg cwe: 398': 4, 'funcArgOrderDifferent cwe: 683': 1, 'memleak cwe: 401': 1, 'bufferAccessOutOfBounds cwe: 788': 3, 'va_end_missing cwe: 664': 1, 'nullPointer cwe: 476': 1, 'shadowFunction cwe: 398': 1, 'arithOperationsOnVoidPointer cwe: 467': 1, 'selfAssignment cwe: 398': 3, 'knownConditionTrueFalse cwe: 570': 7, 'multiCondition cwe: 398': 3, 'incrementboolean cwe: 398': 3, 'clarifyCalculation cwe: 783': 2, 'ignoredReturnValue cwe: 252': 1, 'duplicateExpression cwe: 39